In [ ]:
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
from urllib.error import HTTPError
import pandas as pd
import re


# Списки с данными.
title = []
company = []
price = []
raiting_company = []
raiting_course = []
descr_course = []
url = []

# Цикл на 33 страницы сайта obrazoval.ru
for i in range(1,34):
    site = f'https://obrazoval.ru/courses?direction%5B0%5D=%D0%9F%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%80%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D0%B5&priceFrom=1000&page={i}'
    hdr = {'User-Agent': 'Mozilla/5.0'}
    req = Request(site,headers=hdr)
    page = urlopen(req)
    soup = BeautifulSoup(page, 'lxml')
    link1 = soup.find_all('span',{'class':'ellipsis-3-lines'})
    for num in link1:
        title.append(num.text) # название курса
    link2 = soup.find_all('div', {'class':'q-img q-img--menu'})
    for num1 in link2:
        try:
            if len(title) != len(company): # название компании
                company.append(num1['aria-label'])
        except:
            continue
    link3 = soup.find_all('div',{'class':'b-title__custom l-course__price'})
    for num2 in link3:
        if len(title) != len(price): # Цена курса
            price.append(num2.text)
    link4 = soup.find_all('span', {'class':'owners__info-rating-count'})
    for num3 in link4:
        if len(title) != len(raiting_company): # рейтинг компании
            raiting_company.append(num3.text)
    link5 = soup.find_all('div',{'class':'text-h4 q-mr-xs'})
    for num4 in link5:
        if len(title) != len(raiting_course): # рейтинг курса
            raiting_course.append(num4.text)
    link6 = soup.find_all('div',{'class':'l-course__description ellipsis-4-lines text-dark'})
    for num5 in link6:
        if len(title) != len(descr_course): # краткое описание курса
            descr_course.append(num5.text)
    link7 = soup.find_all('a',{'class':'q-btn q-btn-item non-selectable no-outline q-btn--unelevated q-btn--rectangle q-btn--actionable q-focusable q-hoverable b-btn b-btn--outline l-course__btn'},href=True)
    for num6 in link7:
        if len(title) != len(url): # ссылки подробнее
            url.append(num6['href'])

In [ ]:
# Создаем DataFrame Pandas
df = pd.DataFrame({'Название': title, 'Компания': company, 
'Цена': price, 'РейтингКомп': raiting_company, 
'РейтингКурса': raiting_course, 'Описание': descr_course, 'Ссылки': url})

In [50]:
# Переходим по найденым ссылкам, достаем информацию из подробнее.
skill = []
time_course = []
course_comments = []
try:
    for j in range(len(df["Ссылки"])):
        site = f'https://obrazoval.ru{df["Ссылки"][j]}'
        hdr = {'User-Agent': 'Mozilla/5.0'}
        req = Request(site,headers=hdr)
        page = urlopen(req)
        soup = BeautifulSoup(page, 'lxml')
        lnk_skill = soup.find_all('div', {'class':'l-skills__item'})  # Какие скиллы получаемые за прохождение
        buff = []
        for key in lnk_skill:
            buff.append(re.sub('\d','', key.text))
        skill.append(buff)
        buff = []
        lnk_time = soup.find_all('div', {'class':'flex items-center'}) # Время обучения
        time_course.append(lnk_time[1].text.lstrip()[:-1])
        lnk_comment = soup.find_all('a', {'class':'reviews-card__title'}) # Отзывы о курсе(Только тема)
        for nak in lnk_comment:
            buff.append(nak.text)
        course_comments.append(buff)
        buff = []
except:
    print(f'Ошибка - {df["Ссылки"][j]}')

    


In [51]:
# Дополняем наш DataFrame
df['НавыкиПриобр'] = skill
df['СрокОбуч'] = time_course
df['Коментарии'] = course_comments

In [57]:
# Удаляем не нужные колонки
df.drop(df.columns[[0, 1]], axis = 1, inplace=True)

In [ ]:
# Создаем *.csv
df.to_csv('Course_IT_byKudryai.csv')